# 🔓 RSA Attack Level 2 – Common Modulus & Broadcast

Notebook ini menunjukkan dua serangan RSA saat plaintext yang sama dienkripsi lebih dari sekali:

**1. Common Modulus Attack**
- Modulus `n` sama, eksponen berbeda (`e1`, `e2`)
- Langkah:
  - Hitung `a`, `b` dari `a·e1 + b·e2 = 1`
  - Dekripsi: `m = (c1^a * c2^b) mod n`

**2. Broadcast Attack (Hastad)**
- Eksponen sama (`e = 3`), modulus berbeda (`n1`, `n2`, `n3`)
- Langkah:
  - Gabung ciphertext dengan Chinese Remainder Theorem
  - Ambil akar ke-`e` dari hasilnya untuk memperoleh plaintext


In [62]:
# Import library yang dibutuhkan
from sympy import mod_inverse, isprime, gcd    # Untuk operasi matematika RSA
from math import isqrt                         # Untuk akar integer saat faktorisasi
import random                                  # Untuk generate bilangan acak
from functools import reduce                   # Untuk Chinese Remainder Theorem (CRT)

# ==== Fungsi bantu ====

# Fungsi untuk menghasilkan bilangan prima acak
def generate_prime(bits=8):
    while True:
        p = random.randint(2**(bits - 1), 2**bits - 1)  # Ambil angka acak dalam rentang bits-bit
        if isprime(p):                                  # Cek apakah angka prima
            return p

# Extended Euclidean Algorithm → untuk cari a, b sehingga a*e1 + b*e2 = 1
def extended_gcd(a, b):
    if b == 0:
        return 1, 0
    else:
        x1, y1 = extended_gcd(b, a % b)                 # Rekursif
        x, y = y1, x1 - (a // b) * y1
        return x, y

# Chinese Remainder Theorem → menggabungkan hasil ciphertext dari beberapa modulus
def chinese_remainder_theorem(c_list, n_list):
    N = reduce(lambda a, b: a * b, n_list)              # Total modulus gabungan
    result = 0
    for ci, ni in zip(c_list, n_list):                  # Untuk setiap pasangan ciphertext & modulus
        Ni = N // ni                                    # Modulus bagian lain
        mi = mod_inverse(Ni, ni)                        # Invers mod dari bagian tersebut
        result += ci * Ni * mi                          # Tambahkan kontribusi CRT
    return result % N


In [63]:
# Fungsi generate e kecil (misal di antara 3 sampai 17, ganjil)
def generate_small_e(min_e=3, max_e=17):
    candidates = [x for x in range(min_e, max_e+1) if x % 2 == 1]
    return random.choice(candidates)

# Fungsi generate pesan kecil (m_small) dengan batas max_val
def generate_small_message(max_val=10):
    return random.randint(1, max_val)

# Fungsi generate plaintext (m) dengan batas max_val
def generate_plaintext(max_val=100):
    return random.randint(1, max_val)

# Fungsi encode teks ke integer
def text_to_int(text):
    return int.from_bytes(text.encode('utf-8'), 'big')

# Fungsi decode integer ke teks
def int_to_text(num):
    length = (num.bit_length() + 7) // 8
    return num.to_bytes(length, 'big').decode('utf-8')


In [64]:
# Generate bilangan prima p dan q (bit lebih besar untuk teks, misal 64 bit)
p = generate_prime(bits=64)
q = generate_prime(bits=64)
while p == q:
    q = generate_prime(bits=64)

n = p * q                      # Modulus
phi = (p - 1) * (q - 1)        # Euler's totient function

plaintext = "Daffa Dimas Nova"        # Contoh kalimat plaintext

m = text_to_int(plaintext)     # Encode teks jadi integer

if m >= n:
    raise ValueError("Plaintext terlalu besar untuk modulus n, gunakan modulus lebih besar atau pecah pesan")

# Cari dua eksponen e1 dan e2 yang coprime dengan phi dan antar keduanya
while True:
    e1 = random.randrange(3, phi, 2)
    e2 = random.randrange(3, phi, 2)
    if gcd(e1, phi) == 1 and gcd(e2, phi) == 1 and gcd(e1, e2) == 1:
        break

# Enkripsi dua ciphertext berbeda
c1 = pow(m, e1, n)
c2 = pow(m, e2, n)

# Gunakan Extended Euclidean Algorithm
a, b = extended_gcd(e1, e2)

# Dekripsi menggunakan kombinasi
part1 = pow(mod_inverse(c1, n), -a, n) if a < 0 else pow(c1, a, n)
part2 = pow(mod_inverse(c2, n), -b, n) if b < 0 else pow(c2, b, n)
recovered_common_modulus_int = (part1 * part2) % n

# Decode hasil dekripsi balik ke teks
recovered_text = int_to_text(recovered_common_modulus_int)

In [65]:
e = generate_small_e(3, 17)    # Eksponen kecil random ganjil antara 3-17

# Generate 3 modulus saling coprime
def generate_coprime_moduli(k=3, bits=8):
    primes_used = set()
    n_list = []

    while len(n_list) < k:
        p = generate_prime(bits)
        q = generate_prime(bits)
        if p != q and p not in primes_used and q not in primes_used:
            n_i = p * q
            n_list.append(n_i)
            primes_used.add(p)
            primes_used.add(q)
    return n_list

n_list = generate_coprime_moduli(k=3, bits=8)

# Generate m_small yang memenuhi syarat m_small^e < setiap modulus
while True:
    m_small = generate_small_message(10)
    if all(m_small**e < ni for ni in n_list):
        break

c_list = []
for n_i in n_list:
    c_i = pow(m_small, e, n_i)
    c_list.append(c_i)

# Gabungkan ciphertext dengan CRT
c_combined = chinese_remainder_theorem(c_list, n_list)

# Ambil akar ke-e dari hasil gabungan
approx_root = round(c_combined ** (1 / e))
while approx_root**e > c_combined:
    approx_root -= 1
recovered_hastad = approx_root

In [66]:
# Output Attack 2a
print("===== Common Modulus Scenario =====")
print(f"Modulus n = {n}")
print(f"Exponent e1 = {e1}, Ciphertext c1 = {c1}")
print(f"Exponent e2 = {e2}, Ciphertext c2 = {c2}")
print(f"Recovered plaintext as int: {recovered_common_modulus_int}")
print(f"Recovered plaintext as text: '{recovered_text}'")
print(f"Benar? {recovered_text == plaintext}")


# Output Attack 2b
print("\n===== Hastad Broadcast Scenario =====")
for i in range(3):
    print(f"Modulus n{i+1} = {n_list[i]}")
    print(f"Ciphertext c{i+1} = {c_list[i]}")
print(f"Recovered plaintext: {recovered_hastad} | Benar? {recovered_hastad == m_small}")

===== Common Modulus Scenario =====
Modulus n = 298682073956749933533474672783270325069
Exponent e1 = 7757723851287293887812739831654850857, Ciphertext c1 = 6102929403702765612114979097231045373
Exponent e2 = 295940214703126034732835976847632271399, Ciphertext c2 = 51054735499997699749036793173951085191
Recovered plaintext as int: 90893233425763361207380825484850853473
Recovered plaintext as text: 'Daffa Dimas Nova'
Benar? True

===== Hastad Broadcast Scenario =====
Modulus n1 = 20567
Ciphertext c1 = 2048
Modulus n2 = 24613
Ciphertext c2 = 2048
Modulus n3 = 41917
Ciphertext c3 = 2048
Recovered plaintext: 2 | Benar? True
